In [24]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

learning_rate = 0.001
training_epoch = 15
batch_size = 100

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [27]:
class Model:
    def __init__(self, sess, name):
        self.sess = sess
        self.name = name
        self._build_net()
        
    def _build_net(self):
        with tf.variable_scope(self.name):
            self.training = tf.placeholder(tf.bool)
            self.X = tf.placeholder(tf.float32, shape=[None, 784])
            X_img = tf.reshape(self.X, [-1,28,28,-1])
            self.Y = tf.placeholder(tf.float32, shape=[None, 10])
            
            conv1 = tf.layers.conv2d(inputs = X_img, filters=32,
                                    kernel_size=[3,3], padding='SAME',
                                    activation=tf.nn.relu)
            
            pool1 = tf.layers.pooling2d(input=conv1, pool_sie=[2,2],
                                       padding='SAME', strides=2)
            
            dropout1 = tf.layers.dropout(input=pool1, rate=0.7,
                                        training=self.training)
            
            conv2 = tf.layers.conv2d(inputs = dropout1, filters=64,
                                    kernel_size=[3,3], padding='SAME',
                                    activation=tf.nn.relu)
            
            pool2 = tf.layers.pooling2d(input=conv2, pool_sie=[2,2],
                                       padding='SAME', strides=2)
            
            dropout2 = tf.layers.dropout(input=pool2, rate=0.7,
                                        training=self.training)
            
            conv3 = tf.layers.conv2d(inputs = dropout2, filters=128,
                                    kernel_size=[3,3], padding='SAME',
                                    activation=tf.nn.relu)
            
            pool3 = tf.layers.pooling2d(input=conv3, pool_sie=[2,2],
                                       padding='SAME', strides=2)
            
            dropout3 = tf.layers.dropout(input=pool3, rate=0.7,
                                        training=self.training)
            
            flat = tf.reshape(dropout3, [-1, 128*4*4])
            dense4 = tf.layers.dense(inputs=flat, units=625,
                                    activation=tf.nn.relu)
            
            dropout4 = tf.layers.dropout(inputs=dense4, rate=0.7,
                                        training=self.training)
            
            self.logits = tf.layers.dense(inputs=dropout4, units=10)
        
        self.cost = tf.reduce.mean(tf.nn.softmax_cross_entropy_with_logits(
                    logits=self.logits, labels=self.Y))
        
        self.optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(self.cost)
        
        correct_prediction =tf.equal(tf.argmax(self.logits,1), tf.argmax(self.Y, 1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
        
    def predict(self, x_test, trainig=False):
        return self.sess.run(self.logits, feed_dict={self.X:x_test, \
                                                     self.training:training})
    
    def get_accuracy(self, x_test, y_test, training=False):
        return self.sess.run(self.accuracy,
                            feed_dict={self.X:x_test,
                                      self.Y:y_test,
                                      self.training:training})
    
    def train(self, x_data, y_data, training=True):
        return self.sess.run([self.cost, self.optimizer],
                            feed_dict = {self.X:x_data,
                                        self.Y:y_data,
                                        self.training:training})
            

In [28]:
sess = tf.Session()
models = []
num_models = 7

for m in range(num_models):
    models.append(Model(sess, "model"+str(m)))
    
sess.run(tf.global_variables_initializer())

for epoch in range(training_epoch):
    avg_cost_list = np.zeros(len(models))
    total_batch = int(mnist.train.num_examples/batch_size)
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        
        for m_idx, m in enumerate(models):
            c, _ = m.train(batch_xs, batch_ys)
            avg_cost_list[m_idx] += c/total_batch
            
    print("Epoch:", epoch, "Cost:", avg_cost_list)

test_size = len(mnist.test.labels)
predictions = np.zeros([text_zie, 10])

for m_idx, m in enumerate(models):
    p = m.predict(mnist.test.images)
    predictions += p

ensemble_correct_prediction = tf.equal(tf.argmax(predictions,1),
                                      tf.argmax(mnist.test.lables, 1))

ensemble_accuracy = tf.reduce_mean(tf.cast(ensemble_correct_prediction, tf.float32))

print(sess.run(ensemble_accuracy))

ValueError: The channel dimension of the inputs should be defined. Found `None`.